In [243]:
import nltk
import gensim
import numpy as np
import pandas as pd

import pickle

from nltk.corpus import gutenberg, brown, reuters, stopwords

from collections import Counter, defaultdict
from pprint import pprint
from functools import reduce
from operator import itemgetter
from itertools import product

from utils.nltk_utils import *
from utils.ngrams import *
from utils.eval import get_test_keywords
stop_words = stopwords.words('english')

In [146]:
import traceback
from typing import List, Tuple

from tqdm import tqdm

from utils.grammar_utils import tags_seq_to_symbols
from utils.io import Cache
from utils.ngrams import ngram2str


observed_tags = Cache.load_observed_tags()
terminal_rules = Cache.load_terminal_rules()


def parse_phrases(tt_ngrams) -> Tuple[List, List[List[Tuple]]]:
    global observed_tags, terminal_rules
    
    phrases = []
    phrases_types = []

    if observed_tags is None:
        observed_tags = dict()

    for tt_gram in tt_ngrams:
        symbols = tuple(tags_seq_to_symbols([tag
                                             for _, tag in tt_gram]))

        phrase = tt_gram

        # check if tags phrase has been already observed
        tags_str = ngram2str(symbols)

        if tags_str in observed_tags:
            if observed_tags[tags_str] is not None:
                phrases.append(phrase)
                phrases_types.append(observed_tags[tags_str])

            continue

        p_types_dict = terminal_rules.get(tags_str)

        if p_types_dict:
            p_type = max(p_types_dict, key=lambda k: p_types_dict[k])

            phrases.append(phrase)
            phrases_types.append(p_type)

            observed_tags[tags_str] = p_type

        else:
            observed_tags[tags_str] = None

#     try:
#         Cache.save_observed_tags(observed_tags)

#     except Exception as e:
#         traceback.print_exc(e)

    return phrases_types, phrases


# Preparing corpora

## Load data from NLTK 

In [4]:
sents = gutenberg.sents() + brown.sents() + reuters.sents()

In [5]:
sents = [tuple(s) for s in sents]

words = [w.lower() 
         for s in sents for w in s]

vocab = sorted(list(set(words)))

print('Sents:', len(sents))
print('Words:', len(words))
print('Vocab:', len(vocab))

Sents: 210608
Words: 5503894
Vocab: 87046


In [6]:
# creating indexes

# {sentence: sent_index}
sents_ind = {sents[i]: i for i in range(len(sents))}

# {word: word_index}
vocab_ind = {vocab[j]: j for j in range(len(vocab))}

## Create mapping from words to sentences

In [7]:
stemmer = Stemmer()

stems = [stemmer.stem(word) for word in vocab]

# {word_index: stem}
stems_map = {vocab_ind[word]: stem 
             for word, stem in zip(vocab, stems)}

print('Stems:', len(set(stems)))

Stems: 58858


In [8]:
stem2word = {stem: vocab[ind]
             for ind, stem in stems_map.items()}

## Word2Vec model

In [25]:
# loading
model = gensim.models.Word2Vec.load('data/w2v/CBOW_300_10_all')

In [14]:
sents_stemmed = [[stems_map[vocab_ind[w.lower()]] for w in s] for s in sents]

In [33]:
model = gensim.models.Word2Vec(sentences=sents_stemmed, size=300, window=15, min_count=1, hs=1, negative=0)
model.save('data/w2v/CBOW_300_10_hs_all')

In [34]:
# testing

print(model.wv.most_similar(positive=[stemmer.stem('woman'), stemmer.stem('king')], negative=[stemmer.stem('man')]))
print(model.wv.similarity(stemmer.stem('campus'), stemmer.stem('dormitory')))
print(model.wv.doesnt_match([stemmer.stem(w) for w in "dormitory bring campus".split()]))

[('queen', 0.3553354740142822), ('daughter', 0.35418325662612915), ('esther', 0.3361766040325165), ('husband', 0.3246108889579773), ('absalom', 0.3200189769268036), ('mordecai', 0.31594496965408325), ('samaria', 0.3088769316673279), ('vashti', 0.30666205286979675), ('hebron', 0.298782080411911), ('selleth', 0.29791125655174255)]
0.20989265750352995
bring


In [35]:
w2v = model.wv

In [19]:
del model

## Parsing corpora

In [ ]:
# load

with open('data/cache/stems_phrases', mode='rb') as fp:
    stems_phrases = pickle.load(fp)

In [9]:
parser = NGramsParser()

sents_words_indexes = parser.parse_sents_tokens(sents)

In [147]:
# {stem: Set[sentence_ind]}
stems_phrases = defaultdict(dict)

for i, (s_words, words_indexes) in tqdm(enumerate(zip(sents, sents_words_indexes), start=1)):
    words_ttokens = nltk.pos_tag([w.lower() for w in s_words])
    
    tt_ngrams = [ngr
                 for i in range(2, 5 + 1) 
                 for ngr in n_grams(words_ttokens, i, words_indexes, pad_left=False)]

    types, phrases = parse_phrases(tt_ngrams)

    # format and store phrases
    for t, p in zip(types, phrases):
        phrase_inds = tuple(vocab_ind[token] for token, _ in p)
        
        for word_ind in phrase_inds:
            if vocab[word_ind] not in stop_words:
                stem = stems_map[word_ind]
                
                if not stems_phrases[stem].get(t):
                    stems_phrases[stem][t] = defaultdict(set)
                
                stem_phr_t = stems_phrases[stem][t]
                
                stem_phr_t[phrase_inds].add(sents_ind[s_words])

210608it [06:48, 515.84it/s]


In [148]:
def get_phrases(word, stemmed=True):
    if stemmed:
        stem = word
    else:
        stem = stemmer.stem(word)
        
    phrs = stems_phrases[stem]
    
    if not phrs:
        return {}
    
    phrases_ = defaultdict(set)
    
    for p_type, phr_dict in phrs.items():
        for phrase, sents in phr_dict.items():
            phrase_ = tuple(vocab[ind] for ind in phrase)
            
            phrases_[p_type].add(phrase_)
            
    return phrases_

In [149]:
# save

with open('data/cache/stems_phrases', mode='wb') as fp:
    pickle.dump(stems_phrases, fp)

## Count number of phrases

In [150]:
# data for analysis

phrases_count = defaultdict(dict)

phrases_lengths_count = defaultdict(int)
phrases_types_count = defaultdict(int)

stems_phrases_count = defaultdict(dict)

for stem, stem_phrases_dict in stems_phrases.items():
    for phrase_type, phrases_dict in stem_phrases_dict.items():
#         stem_phrase_type_count = sum([len(s_ids) for s_ids in phrases_dict.values()])
        for phrase_tuple, s_ids in phrases_dict.items():
            phrase_len = len(phrase_tuple)
            phrase_sents_count = len(s_ids)
            
            if not stems_phrases_count[stem].get(phrase_len):
                stems_phrases_count[stem][phrase_len] = defaultdict(int)
                
            # count number of such phrase type for given stem
            stems_phrases_count[stem][phrase_len][phrase_type] += phrase_sents_count 

            # count total number of such phrase type
            phrases_count[phrase_len][phrase_type] = phrases_count[phrase_len].get(phrase_type, 0) + phrase_sents_count

            # count total number of all phrases
            phrases_lengths_count[phrase_len] += phrase_sents_count
            phrases_types_count[phrase_type] += phrase_sents_count

all_phrases_count = sum(phrases_types_count.values())

In [151]:
phrases_count

defaultdict(dict,
            {2: {'ADJP': 288142,
              'ADVP': 418156,
              'NP': 1985844,
              'PP': 3328,
              'VP': 66991},
             3: {'ADJP': 44770,
              'ADVP': 31064,
              'NP': 1353053,
              'PP': 7554,
              'VP': 8215},
             4: {'ADJP': 4179, 'ADVP': 223, 'NP': 429226},
             5: {'ADJP': 245, 'NP': 53025}})

In [291]:
WORD = 'strive'

stems_phrases_count[stemmer.stem(WORD)]

{2: defaultdict(int, {'ADJP': 12, 'ADVP': 8, 'NP': 28, 'VP': 9}),
 3: defaultdict(int, {'ADJP': 1, 'NP': 12, 'VP': 1}),
 4: defaultdict(int, {'NP': 4})}

In [292]:
def phrases_model(word, stemmed=True):
    """
        Return sorted log probs of phrase types and length for the word 
    """
    if not stemmed:
        stem = stemmer.stem(word)
    else:
        stem = word
    
    if not stems_phrases_count.get(stem):
        return []
    
    stem_phrases = stems_phrases_count[stem]
    
    phrases_probs = []
    
    for p_length, p_types_count in stem_phrases.items():
        for p_type, p_type_count in p_types_count.items():
            prob_len = phrases_lengths_count[p_length] / all_phrases_count
            
            prob_len_type = phrases_count[p_length][p_type] / phrases_lengths_count[p_length]
            
            prob_word_len_type = p_type_count / phrases_count[p_length][p_type]
            
            log_prob = sum(np.log(p) for p in [prob_word_len_type, prob_len_type, prob_len])
            
            phrases_probs.append(((p_type, p_length), log_prob))
    
    phrases, scores = [], []
    for phr, s in sorted(phrases_probs, key=itemgetter(1), reverse=True):
        phrases.append(phr)
        scores.append(s)
    
#     scores = np.exp(scores)
#     scores /= np.max(scores)
    
    return list(zip(phrases, scores))

phrases_model(WORD, stemmed=False)

[(('NP', 2), -12.029594340781607),
 (('ADJP', 2), -12.87689220116881),
 (('NP', 3), -12.876892201168811),
 (('VP', 2), -13.164574273620593),
 (('ADVP', 2), -13.282357309276973),
 (('NP', 4), -13.97550448983692),
 (('VP', 3), -15.361798850956813),
 (('ADJP', 3), -15.361798850956813)]

## Scoring phrases (experiments)

In [294]:
LENGTH_PEN = {
    2: 1.75,
    3: 1.25,
    4: 0.75,
    5: 0.25
}

REVERSE = False
SCORE_SIGN = -1
TOP_N = 5

P_TYPE_SCORE = dict(phrases_model(WORD, stemmed=False))

phrases_ = get_phrases(WORD, stemmed=False)

all_scored_phrases = defaultdict(dict)

for p_type, phrases in phrases_.items():
    phrases_ = list(phrases)
    
    scores = np.array(model.score([[stemmer.stem(w) for w in phr] 
                                   for phr in phrases_]))

    for phr, sc in zip(phrases_, scores):
        if not all_scored_phrases[p_type].get(len(phr)):
            all_scored_phrases[p_type][len(phr)] = []
        
        score = SCORE_SIGN * (sc + P_TYPE_SCORE[(p_type, len(phr))]) * LENGTH_PEN[len(phr)] 
        
        all_scored_phrases[p_type][len(phr)].append((phr, score))

In [295]:
for p_type, scored_phrases in all_scored_phrases.items():
    print('%s\n' % p_type)
    for n in scored_phrases:
        nps = scored_phrases[n]
        best_phr = sorted(nps, key=itemgetter(1), reverse=REVERSE)[:TOP_N]

        best_phr = [(' '.join(p), s) for p, s in best_phr]

        print(n)
        pprint(best_phr)
        print()

NP

2
[('strive against', 45.37863693436525),
 ('also strive', 45.6956552048578),
 ('thou strive', 51.02504954537294),
 ('economic strivings', 51.26184067925355),
 ('the strivings', 52.57374654015443)]

3
[('the time striving', 42.21950544921492),
 ('no man strive', 42.35136284438704),
 ('dost thou strive', 44.11854281035994),
 ('a striving after', 44.736132760921464),
 ('a critic striving', 47.06441893188094)]

4
[('social and economic strivings', 26.75404972739356),
 ('than a critic striving', 32.95443148322242),
 ('the potsherd strive with', 34.64571375556129),
 ('one powerful nation strives', 36.17675871558326)]

ADVP

2
[('strive against', 47.570972129232146),
 ('strive with', 47.65035311830685),
 ('strive for', 56.87991182459042),
 ('striving after', 60.76540462625668),
 ('strive at', 72.63408748758481)]

VP

2
[('striving against', 47.36485181683348),
 ('striving with', 47.444232805908186),
 ('striving for', 56.673791512191755),
 ('strive not', 57.704012020767195),
 ('striving i

In [258]:
# group different length together

candidate_phrases = defaultdict(list)

for p_type, scored_phrases in all_scored_phrases.items():
    for n in scored_phrases:
        nps = scored_phrases[n]
        best_phr = sorted(nps, key=itemgetter(1), reverse=REVERSE)[:TOP_N]

        best_phr = [(' '.join(p), s) for p, s in best_phr]
        
        candidate_phrases[p_type].extend(best_phr)
        
    # TODO: filter similar phrases and phrases with keywords
    candidate_phrases[p_type] = sorted(candidate_phrases[p_type], key=itemgetter(1), reverse=REVERSE)[:TOP_N]
    
candidate_phrases

defaultdict(list,
            {'ADJP': [('costly and taking', 43.71989976637133),
              ('then taking', 46.55225528239214),
              ('not taking', 48.7940656042),
              ('take ye good', 49.20127879850634),
              ('once taking', 49.73397696970903)],
             'ADVP': [('taking over', 36.99610413129595),
              ('take over', 36.99610413129595),
              ('take of', 41.23225248868731),
              ('of take', 41.23225248868731),
              ('taking of', 41.23225248868731)],
             'NP': [('this take constant vigilance ."', 13.749093078121678),
              ("an employee's garden club take", 14.565914176449315),
              ('the same time taking', 21.42702209067359),
              ('taking place', 24.3107733921767),
              ('take places', 24.3107733921767)],
             'PP': [('on its takings', 43.761827459264865)],
             'VP': [('take away', 29.890197170588603),
              ('takes away', 29.890197170588603),
  

## Implementation of scoring phrases 

In [263]:
# TODO: add probabilistic CFG

LENGTH_PEN = {
    2: 1.75,
    3: 1.25,
    4: 0.75,
    5: 0.5
}

REVERSE = False
SCORE_SIGN = -1
TOP_N = 5

def get_scored_phrases(word, include_scores=True):
    P_TYPE_SCORE = dict(phrases_model(word, stemmed=False))

    phrases_ = get_phrases(word, stemmed=False)

    all_scored_phrases = defaultdict(dict)

    for p_type, phrases in phrases_.items():
        phrases_ = list(phrases)

        scores = np.array(model.score([[stemmer.stem(w) for w in phr] 
                                       for phr in phrases_]))

        for phr, sc in zip(phrases_, scores):
            if not all_scored_phrases[p_type].get(len(phr)):
                all_scored_phrases[p_type][len(phr)] = []

            score = SCORE_SIGN * (sc + P_TYPE_SCORE[(p_type, len(phr))]) * LENGTH_PEN[len(phr)] 

            all_scored_phrases[p_type][len(phr)].append((phr, score))

    # group different length together

    candidate_phrases = defaultdict(list)

    for p_type, scored_phrases in all_scored_phrases.items():
        for n in scored_phrases:
            nps = scored_phrases[n]
            best_phr = sorted(nps, key=itemgetter(1), reverse=REVERSE)[:TOP_N]

            best_phr = [(' '.join(p), s) for p, s in best_phr]

            candidate_phrases[p_type].extend(best_phr)

        # TODO: filter similar phrases and phrases with keywords
        candidate_phrases[p_type] = sorted(candidate_phrases[p_type], key=itemgetter(1), reverse=REVERSE)[:TOP_N]
        
        if not include_scores:
            candidate_phrases[p_type] = [p for p, _ in candidate_phrases[p_type]]
            
    return candidate_phrases

## Working with keywords

In [239]:
kws = get_test_keywords('data/lingualeo_words.csv')

print('Keywords: ', len(kws))

Keywords:  874


In [286]:
TEST_SIZE = 5
np.random.seed = 0

all_kws_found = False

while not all_kws_found:    
    test_kws = list(np.random.choice(list(kws), size=TEST_SIZE, replace=False))

    print('Test keywords:', test_kws)

    test_kws = [stemmer.stem(kw) for kw in test_kws]
    kws_phrases = [stems_phrases.get(kw) for kw in test_kws]
    print('Test keywords stems:', test_kws)

    all_kws_found = all(kws_phrases)
    
    print('All kws found:', all_kws_found)
    print()
    

Test keywords: ['eagerly', 'supply', 'arise', 'feed', 'emergence']
Test keywords stems: ['eager', 'suppli', 'aris', 'feed', 'emerg']
All kws found: True



In [290]:
pprint(', '.join(['eagerly', 'supply', 'arise', 'feed', 'emergence']))

'eagerly, supply, arise, feed, emergence'


In [287]:
def cluster_keywords(kws):
    pairs = []

    for i, kw in enumerate(kws):
        cp = list(kws)
        cp.remove(kw)

        dsts = w2v.distances(kw, cp)
#         print(kw, dsts)
        max_ind = np.argmin(dsts)
#         print(i, max_ind)
        sim_kw = kws[max_ind + (1 if i <= max_ind else 0)]
#         print(kw, sim_kw)

        pair = (kw, sim_kw)

        if (sim_kw, kw) in pairs:
            pair = (sim_kw, kw)

        pairs.append(pair)

    final_clusters = []
    
    kws_ = list(kws)
    
    for kws_pair, cnt in Counter(pairs).items():
        if cnt > 1:
            final_clusters.append(kws_pair)
            [kws_.remove(w) for w in kws_pair]
        
    return final_clusters + [(kw,) for kw in kws_]

cluster_keywords(test_kws)

[('suppli', 'feed'), ('aris', 'emerg'), ('eager',)]

In [282]:
for kws_tuple in cluster_keywords(test_kws):
    sents = gen_sents_candidates(kws_tuple, [get_scored_phrases(kw, include_scores=False) for kw in kws_tuple])
    
    ranked_sents = rank_sents(sents)
    
    print(kws_tuple, len(sents))
    pprint(ranked_sents)
    print()

('harsh', 'nutshel') 15060
[(['so', 'harsh', ',', 'said', 'harsh'], -32.27874),
 (['so', 'harsh', 'for', 'that', 'harsh'], -32.34414),
 (['nutshel', ',"', 'said', 'harsh', ','], -32.487923),
 (['nutshel', ',"', ',', 'said', 'harsh'], -32.487923),
 (['a', 'nutshel', ',"', 'said', 'harsh'], -32.695766),
 (['said', 'harsh', 'a', 'nutshel', ',"'], -32.69577),
 (['grate', 'so', 'harsh', 'that', 'harsh'], -33.295624),
 (['said', 'harsh', 'in', 'a', 'harsh', 'voic'], -34.27955),
 (['in', 'a', 'harsh', 'voic', 'said', 'harsh'], -34.27955),
 (['the', 'harsh', 'voic', 'of', 'said', 'harsh'], -34.46181)]

('essenti', 'implicit') 121171
[(['the', 'essenti', 'foreign', 'exchang', 'requir'], -28.382555),
 (['all', 'the', 'essenti', 'are', 'essenti'], -28.967785),
 (['essenti', 'within', 'the', 'are', 'essenti'], -29.271837),
 (['essenti', 'of', 'a', 'is', 'essenti'], -29.710833),
 (['not', 'essenti', ',', 'are', 'essenti'], -30.0416),
 (['essenti', 'not', ',', 'are', 'essenti'], -30.0416),
 (['essen

In [281]:
sents_rules = Cache.load_terminal_rules_sents()

def rank_sents(sents, top_n=10):
    sents = [[stems_map.get(vocab_ind.get(w, w), w) 
              for w in s.split(' ')] 
             for s in sents
            if len(s.split(' ')) > 4]
    return sorted(zip(sents, model.score(sents)), key=itemgetter(1), reverse=True)[:top_n]
    
def combine_elements(*args):
    return list(product(*args, repeat=1))

def gen_sents_candidates(kws, kws_phrases):
    marks = {'COMMA': [","],
             'COLON': [":"],
             'SEMICOLON': [";"],
             # 'DOT': ["."],
             'QUESTION': ["?"],
             'EXCLAM': ["!"],
             'DASH': ["-"]}

    sents = set()

    phrases = defaultdict(dict)

    for kw, kw_phrases in zip(kws, kws_phrases):
        for p_type in kw_phrases:
            phrases[p_type][kw] = kw_phrases[p_type]

    for sents_rule in sents_rules:
        sent_symbols = str2ngram(sents_rule)
        
        sent_phrases = []
        
        # form lists of each phrase
        for p_type in sent_symbols:
            if p_type not in marks:
                sent_phrases.append([phrases[p_type].get(kw) 
                                     for kw in kws
                                    if phrases[p_type].get(kw)])
            else:
                sent_phrases.append(marks.get(p_type, []))
            
        # combine phrases of keywords
        kws_phrases_product = combine_elements(*sent_phrases)
        
        # combine sents
        for kws_comb in kws_phrases_product:
            sents_candidates = [' '.join(s) for s in combine_elements(*kws_comb)]
            
            sents.update(sents_candidates)
    
    return sents

## Using processed corpora

In [ ]:
# test

t = 'grievance'

s = stemmer.stem(t)
# stems_phrases[s]
vocab_ind['grievance']

In [ ]:
# test

t = 'grievance'

s = stemmer.stem(t)

ss = []
for k, v in stems_phrases[s]['NP'].items():
    ss.extend([s_id for s_id in v])
    
Counter(ss)